# Include a for loop within functions

# Countsnps
    - input: query and reference fasta file
    - output: matrix (snp count)
# calcpdistance 
    - input: query and reference fasta file
    - output: matrix (pairwise distance)
# Assigngenotypes 
    - input: matrix (pairwise distance)
    - output: csv file (user sample names, reference name, the corresponding pairwise distance)

In [2]:

readFasta <- function(fastaFile) {
  # Read all lines from the FASTA file
  lines <- readLines(fastaFile)
  
  # Initialize lists to store sequences and their headers
  seqList <- list() # Empty list
  headerList <- c() # Empty vector
  
  # Temporary storage for the current sequence being read
  currentSeq <- NULL
  
  # Iterate through each line of the FASTA file
  for (line in lines) {
    if (startsWith(line, ">")) {
      # If currentSeq is not NULL, it means we've finished reading a sequence
      # Add it to seqList
      if (!is.null(currentSeq)) {
        seqList[[length(seqList) + 1]] <- paste(currentSeq, collapse = "")
      }
      # Reset currentSeq for the next sequence
      currentSeq <- c()
      # Add the header (without the ">" character) to headerList
      headerList <- c(headerList, substring(line, 2))
    } else {
      # If the line is not a header, it's part of the current sequence
      # Convert it to uppercase and add it to currentSeq
      currentSeq <- c(currentSeq, toupper(line))
    }
  }
  
  # After the loop, add the last sequence to seqList if it exists
  if (!is.null(currentSeq)) {
    seqList[[length(seqList) + 1]] <- paste(currentSeq, collapse = "")
  }
  
  # Return a list containing the sequences and their corresponding headers
  return(list(sequences = seqList, headers = headerList))
}
                   
    

In [3]:
# Count the SNPs
countSNPs <- function(pathToRef, pathToQuery) {
  # Read reference sequences and headers
  fastaRef <- readFasta(pathToRef)
  refs <- fastaRef$sequences
  refHeaders <- fastaRef$headers
  
  # Read query sequences and headers
  fastaQuery <- readFasta(pathToQuery)
  queries <- fastaQuery$sequences
  queryHeaders <- fastaQuery$headers
  
  # Initialize matrix to store SNP counts
  snpMatrix <- matrix(nrow = length(queries), ncol = length(refs))
  
  # Convert reference sequences to character vectors
  refSeqs <- lapply(refs, function(ref) unlist(strsplit(ref, split = "")))
  
  # Iterate over each query sequence
  for (q in seq_along(queries)) {
    query <- queries[[q]]
    queryHeader <- queryHeaders[[q]]
    
    # Convert query sequence to character vector
    queryChars <- unlist(strsplit(query, split = ""))
    
    # Iterate over each reference sequence
    for (i in seq_along(refSeqs)) {
      refChars <- refSeqs[[i]]
      
      # Ensure both sequences are of the same length for comparison
      if (length(refChars) == length(queryChars)) {
        # Count the differences (SNPs)
        snpMatrix[q, i] <- sum(refChars != queryChars)
      } else {
        snpMatrix[q, i] <- NA  # Mark as NA if sequences are of different lengths (I have a problem with this, how do we actually test this?(perform inhouse alignment))
      }
    }
  }
  
  # Set row and column names
  rownames(snpMatrix) <- queryHeaders
  colnames(snpMatrix) <- refHeaders
  
  # Return SNP counts matrix
  return(snpMatrix)
}

###### - This section of countSNPs needs further discussion with other members

`if (length(refChars) == length(queryChars)) {
        # Count the differences (SNPs)
        snpMatrix[q, i] <- sum(refChars != queryChars)
      } else {
        snpMatrix[q, i] <- NA
    
` 

#Mark as NA if sequences are of different lengths (I have a problem with this, how do we actually test this?(perform inhouse alignment))

In [11]:
calcPDistance <- function(pathToRef, pathToQuery) {
  # Read reference sequences and headers
  fastaRef <- readFasta(pathToRef)
  refs <- fastaRef$sequences
  refHeaders <- fastaRef$headers
  
  # Read query sequences and headers
  fastaQuery <- readFasta(pathToQuery)
  queries <- fastaQuery$sequences
  queryHeaders <- fastaQuery$headers
  
  # Initialize matrix to store p-distances
  pDistancesMatrix <- matrix(nrow = length(queries), ncol = length(refs))
  
  # Iterate over each query sequence
  for (q in seq_along(queries)) {
    query <- queries[[q]]
    queryHeader <- queryHeaders[[q]]
    
    # Count SNPs between the query and each reference sequence
    snpCounts <- countSNPs(pathToRef, pathToQuery)
    
    # Calculate p-distance for each reference sequence
    for (i in seq_along(refHeaders)) {
      ref <- refs[[i]]
        #print(ref) Debugging
      snpCount <- snpCounts[q, i]  # SNP count between query and reference
      
      # Calculate p-distance
      pDistancesMatrix[q, i] <- snpCount / nchar(ref)
    }
  }
  
  # Set row and column names
  rownames(pDistancesMatrix) <- queryHeaders
  colnames(pDistancesMatrix) <- refHeaders
  
  return(pDistancesMatrix)
}


In [12]:
# Count snps (query vs ref)
countSNPs('test_align.fasta','test_align_1_usernames.fasta')


,97816_A8,98103_B48,97412_C43,GQ323774.1_RVC10genome,95114_C10
sample43,156,182,0,101,90
sample10,164,173,101,0,28
sample10,157,190,90,28,0


In [13]:
calcPDistance('test_align.fasta','test_align_1_usernames.fasta')


,97816_A8,98103_B48,97412_C43,GQ323774.1_RVC10genome,95114_C10
sample43,0.3489933,0.4071588,0.0000000,0.22595078,0.20134228
sample10,0.3668904,0.3870246,0.2259508,0.00000000,0.06263982
sample10,0.3512304,0.4250559,0.2013423,0.06263982,0.00000000


In [7]:
# Evaluate each row entry for assigning the genotypes
# In each row check for the lowest value and compare it to a given threshold and assign thatgiven row the corresponding column header
# In progress